In [1]:
import os
import cv2
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from labelme import utils
from skimage.feature import peak_local_max

# Point Detection

In [2]:
def blur(image, ks=7):
    kernel_size = (ks, ks)
    blurred_image = cv2.GaussianBlur(image, kernel_size, sigmaX=0)
    return blurred_image

In [3]:
def norm_0_1(mask):    
    x_min = np.min(mask)
    x_max = np.max(mask)
    
    new_mask = (mask-x_min)/(x_max-x_min)
    
    return new_mask

def get_dotsmap(den_map, min_dis=14, thres=0.8, light_thres=60):
    
    if np.max(den_map) < thres:
        return []
    
    den_map_norm = norm_0_1(den_map)
    
    x_y = peak_local_max(
        den_map, 
        min_distance = min_dis,
        threshold_abs = thres,
    )
    
    dots_map = np.zeros(den_map.shape)
    dots_map[x_y[:, 0].tolist(), x_y[:, 1].tolist()] = 1
    
    return np.array(dots_map, np.uint8)

In [4]:
def get_json(img_path):
    img = np.array(Image.open(img_path))
    lbl = get_dotsmap(img)
    w, h = np.where(lbl != 0)
    
    points = [[h[i], w[i]] for i in range(len(h))]
    points = np.array(points, np.int32).tolist()
    shapes = [{"label": 'atom', "points": [item], "group_id": None, "shape_type": "point", "flags": {}} for item in points]
    imagePath = img_path.split('/')[-1]
    imageData = utils.img_arr_to_b64(img).decode('utf-8')
    imageHeight, imageWidth = img.shape
    
    json_data = {
        'version': '5.0.1',
        'flags': {},
        'shapes': shapes,
        'imagePath': imagePath,
        'imageData': imageData,
        'imageHeight': imageHeight,
        'imageWidth': imageWidth
    }
    
    return json_data

In [5]:
def save_json(img_path):
    json_data = get_json(img_path)
    
    with open(img_path.replace('.jpg', '.json'), 'w') as f:
        json.dump(json_data, f)

# Save Json

In [6]:
img_lst = glob.glob('../data/*.jpg') 

for item in img_lst:
    save_json(item)